In [1]:
from monai.utils import first
from monai.transforms import (
    AddChanneld,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandSpatialCropd,
    ScaleIntensityRanged,
    Spacingd,
    Resized,
    NormalizeIntensityd,
    ToTensord,
    ScaleIntensityd,
    RandRotated,
    RandZoomd,
    RandGaussianNoised,
    RandFlipd,
    RandAffined,
    Flip,
    Rand2DElasticd,
    Rand3DElasticd,
    Rotate90d
)



from monai.data import DataLoader, Dataset ,NiftiSaver
import matplotlib.pyplot as plt
import os
import glob
import numpy as np
import os
import torch
import nibabel as nib
#from tqdm import tqdm
from glob import glob 
import pandas as pd
import nibabel as nib
import time

input_path = "C:/Users/85298/AI/CT_image/202304/224_160_48/ori/"
output_locaton = "C:/Users/85298/AI/CT_image/202304/224_160_48/"

def save_nifti(in_image, out, index = 0):
    # Convert the torch tensors into numpy array
    volume = np.array(in_image.detach().cpu()[0, :, :, :], dtype=np.float32)

    volume = np.reshape (volume, (224,160,48))
    # Convert the numpy array into nifti file
    volume = nib.Nifti1Image(volume, np.eye(4))      
        
    volume.to_filename( out )
    
    print(f'patient_generated_{index} is saved', end='\r')
    
    
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)    


C:\Users\85298\.conda\envs\tensorflow\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ori_1 = Compose(
    [
         LoadImaged(keys=["image"]),
        AddChanneld(keys=["image"]),
        ToTensord(keys=["image"]),
    ]
)


flip_xy = Compose(
    [
         LoadImaged(keys=["image"]),
        AddChanneld(keys=["image"]),
        RandFlipd(keys=["image"],spatial_axis=1,prob=1),
        RandFlipd(keys=["image"],spatial_axis=0,prob=1),
        ToTensord(keys=["image"]),
    ]
)




shear_x1 = Compose(
    [
        LoadImaged(keys=["image"]),
        AddChanneld(keys=["image"]),
        RandAffined(keys=["image"],prob=1,shear_range=(-0.2,0),mode=["bilinear"],padding_mode='border'),
        ToTensord(keys=["image"]),
    ]
)


shear_x2 = Compose(
    [
        LoadImaged(keys=["image"]),
        AddChanneld(keys=["image"]),
        RandAffined(keys=["image"],prob=1,shear_range=(0.2,0),mode=["bilinear"],padding_mode='border'),
        ToTensord(keys=["image"]),
    ]
)



elast_1 = Compose(
    [
        LoadImaged(keys=["image"]),
        AddChanneld(keys=["image"]),
        Rand3DElasticd(keys='image',sigma_range=(5,7),magnitude_range=(50,150), prob=1,mode=["bilinear"]),        
        ToTensord(keys=["image"]),
    ]
)



noise_1 = Compose(
    [
        LoadImaged(keys=["image"]),
        AddChanneld(keys=["image"]),
        RandGaussianNoised(keys='image', prob=1),     
        ToTensord(keys=["image"]),
    ]
)


shear_x1n = Compose(
    [
        LoadImaged(keys=["image"]),
        AddChanneld(keys=["image"]),
        RandAffined(keys=["image"],prob=1,shear_range=(-0.2,0),mode=["bilinear"],padding_mode='border'),
        RandGaussianNoised(keys='image', prob=1),
        ToTensord(keys=["image"]),
    ]
)


shear_x2n = Compose(
    [
        LoadImaged(keys=["image"]),
        AddChanneld(keys=["image"]),
        RandAffined(keys=["image"],prob=1,shear_range=(0.2,0),mode=["bilinear"],padding_mode='border'),
        RandGaussianNoised(keys='image', prob=1),
        ToTensord(keys=["image"]),
    ]
)



elast_n = Compose(
    [
        LoadImaged(keys=["image"]),
        AddChanneld(keys=["image"]),
        Rand3DElasticd(keys='image',sigma_range=(5,7),magnitude_range=(50,150), prob=1,mode=["bilinear"]),
        RandGaussianNoised(keys='image', prob=1),
        ToTensord(keys=["image"]),
    ]
)




flip_xyn = Compose(
    [
         LoadImaged(keys=["image"]),
        AddChanneld(keys=["image"]),
        RandFlipd(keys=["image"],spatial_axis=1,prob=1),
        RandFlipd(keys=["image"],spatial_axis=0,prob=1),
        RandGaussianNoised(keys='image', prob=1),
        ToTensord(keys=["image"]),
    ]
)


flip_xye = Compose(
    [
         LoadImaged(keys=["image"]),
        AddChanneld(keys=["image"]),
        RandFlipd(keys=["image"],spatial_axis=1,prob=1),
        RandFlipd(keys=["image"],spatial_axis=0,prob=1),
        Rand3DElasticd(keys='image',sigma_range=(5,7),magnitude_range=(50,150), prob=1,mode=["bilinear"]),
        ToTensord(keys=["image"]),
    ]
)



flip_xyen = Compose(
    [
         LoadImaged(keys=["image"]),
        AddChanneld(keys=["image"]),
        RandFlipd(keys=["image"],spatial_axis=1,prob=1),
        RandFlipd(keys=["image"],spatial_axis=0,prob=1),
        Rand3DElasticd(keys='image',sigma_range=(5,7),magnitude_range=(50,150), prob=1,mode=["bilinear"]),
        RandGaussianNoised(keys='image', prob=1),
        ToTensord(keys=["image"]),
    ]
)





C:\Users\85298\.conda\envs\tensorflow\lib\site-packages\monai\utils\deprecate_utils.py:107: FutureWarning: <class 'monai.transforms.utility.array.AddChannel'>: Class `AddChannel` has been deprecated since version 0.8. please use MetaTensor data type and monai.transforms.EnsureChannelFirst instead.
  warn_deprecated(obj, msg, warning_category)


In [5]:
#1min
start = time.time()
name = "flip_xy"
output_path = output_locaton + name


number_runs = 1
imglist = sorted(glob(input_path+"*.nii.gz"))
train_files =  [{"image": image} for image in zip(imglist)]
print(train_files)

[{'image': ('C:/Users/85298/AI/CT_image/202304/224_160_48/ori\\2002_ct.nii.gz',)}, {'image': ('C:/Users/85298/AI/CT_image/202304/224_160_48/ori\\5006_ct.nii.gz',)}, {'image': ('C:/Users/85298/AI/CT_image/202304/224_160_48/ori\\6007_ct.nii.gz',)}]


In [3]:
#1min
start = time.time()
name = "flip_xy"
output_path = output_locaton + name


number_runs = 1
imglist = sorted(glob(input_path+"*.nii.gz"))
train_files =  [{"image": image} for image in zip(imglist)]

for i in range(number_runs):

    output = os.path.join(output_path)
    check_ds = Dataset(data=train_files, transform= flip_xy )
    check_loader = DataLoader(check_ds, batch_size=1)
    check_data = first(check_loader)
    for index, patient in enumerate(check_loader):
        
        aa = str(train_files[index])
        ab = aa.split("\\")
        ac = ab[2].split(".")
        ad = ac[0]
        
        
        output2 = output + "/" + name + "_" + ad + ".nii.gz"
        save_nifti(patient['image'], output2, index)
#        print(output2)
        print(index)
        
    print(f'step {i} done')
elapsed = time.time()-start
print (f'Training time: {hms_string(elapsed)}')    

0atient_generated_0 is saved
1atient_generated_1 is saved
2atient_generated_2 is saved
3atient_generated_3 is saved
4atient_generated_4 is saved
5atient_generated_5 is saved
6atient_generated_6 is saved
7atient_generated_7 is saved
8atient_generated_8 is saved
9atient_generated_9 is saved
10tient_generated_10 is saved
11tient_generated_11 is saved
12tient_generated_12 is saved
13tient_generated_13 is saved
14tient_generated_14 is saved
15tient_generated_15 is saved
16tient_generated_16 is saved
17tient_generated_17 is saved
18tient_generated_18 is saved
19tient_generated_19 is saved
20tient_generated_20 is saved
21tient_generated_21 is saved
22tient_generated_22 is saved
23tient_generated_23 is saved
24tient_generated_24 is saved
25tient_generated_25 is saved
26tient_generated_26 is saved
27tient_generated_27 is saved
28tient_generated_28 is saved
29tient_generated_29 is saved
30tient_generated_30 is saved
31tient_generated_31 is saved
32tient_generated_32 is saved
33tient_generated_33

In [4]:
#1min
start = time.time()
name = "shear_x1"
output_path = output_locaton + name


number_runs = 1
imglist = sorted(glob(input_path+"*.nii.gz"))
train_files =  [{"image": image} for image in zip(imglist)]

for i in range(number_runs):

    output = os.path.join(output_path)
    check_ds = Dataset(data=train_files, transform=shear_x1)
    check_loader = DataLoader(check_ds, batch_size=1)
    check_data = first(check_loader)
    for index, patient in enumerate(check_loader):
        
        aa = str(train_files[index])
        ab = aa.split("\\")
        ac = ab[2].split(".")
        ad = ac[0]
        
        
        output2 = output + "/" + name + "_" + ad + ".nii.gz"
        save_nifti(patient['image'], output2, index)
#        print(output2)
        print(index)
        
    print(f'step {i} done')
elapsed = time.time()-start
print (f'Training time: {hms_string(elapsed)}')      

0atient_generated_0 is saved
1atient_generated_1 is saved
2atient_generated_2 is saved
3atient_generated_3 is saved
4atient_generated_4 is saved
5atient_generated_5 is saved
6atient_generated_6 is saved
7atient_generated_7 is saved
8atient_generated_8 is saved
9atient_generated_9 is saved
10tient_generated_10 is saved
11tient_generated_11 is saved
12tient_generated_12 is saved
13tient_generated_13 is saved
14tient_generated_14 is saved
15tient_generated_15 is saved
16tient_generated_16 is saved
17tient_generated_17 is saved
18tient_generated_18 is saved
19tient_generated_19 is saved
20tient_generated_20 is saved
21tient_generated_21 is saved
22tient_generated_22 is saved
23tient_generated_23 is saved
24tient_generated_24 is saved
25tient_generated_25 is saved
26tient_generated_26 is saved
27tient_generated_27 is saved
28tient_generated_28 is saved
29tient_generated_29 is saved
30tient_generated_30 is saved
31tient_generated_31 is saved
32tient_generated_32 is saved
33tient_generated_33

In [5]:
#1min
start = time.time()
name = "shear_x2"
output_path = output_locaton + name


number_runs = 1
imglist = sorted(glob(input_path+"*.nii.gz"))
train_files =  [{"image": image} for image in zip(imglist)]

for i in range(number_runs):

    output = os.path.join(output_path)
    check_ds = Dataset(data=train_files, transform=shear_x2)
    check_loader = DataLoader(check_ds, batch_size=1)
    check_data = first(check_loader)
    for index, patient in enumerate(check_loader):
        
        aa = str(train_files[index])
        ab = aa.split("\\")
        ac = ab[2].split(".")
        ad = ac[0]
        
        
        output2 = output + "/" + name + "_" + ad + ".nii.gz"
        save_nifti(patient['image'], output2, index)
#        print(output2)
        print(index)
        
    print(f'step {i} done')
elapsed = time.time()-start
print (f'Training time: {hms_string(elapsed)}')  

0atient_generated_0 is saved
1atient_generated_1 is saved
2atient_generated_2 is saved
3atient_generated_3 is saved
4atient_generated_4 is saved
5atient_generated_5 is saved
6atient_generated_6 is saved
7atient_generated_7 is saved
8atient_generated_8 is saved
9atient_generated_9 is saved
10tient_generated_10 is saved
11tient_generated_11 is saved
12tient_generated_12 is saved
13tient_generated_13 is saved
14tient_generated_14 is saved
15tient_generated_15 is saved
16tient_generated_16 is saved
17tient_generated_17 is saved
18tient_generated_18 is saved
19tient_generated_19 is saved
20tient_generated_20 is saved
21tient_generated_21 is saved
22tient_generated_22 is saved
23tient_generated_23 is saved
24tient_generated_24 is saved
25tient_generated_25 is saved
26tient_generated_26 is saved
27tient_generated_27 is saved
28tient_generated_28 is saved
29tient_generated_29 is saved
30tient_generated_30 is saved
31tient_generated_31 is saved
32tient_generated_32 is saved
33tient_generated_33

In [6]:
#5min
start = time.time()
name = "elast_1"
output_path = output_locaton + name


number_runs = 1
imglist = sorted(glob(input_path+"*.nii.gz"))
train_files =  [{"image": image} for image in zip(imglist)]

for i in range(number_runs):

    output = os.path.join(output_path)
    check_ds = Dataset(data=train_files, transform=elast_1)
    check_loader = DataLoader(check_ds, batch_size=1)
    check_data = first(check_loader)
    for index, patient in enumerate(check_loader):
        
        aa = str(train_files[index])
        ab = aa.split("\\")
        ac = ab[2].split(".")
        ad = ac[0]
        
        
        output2 = output + "/" + name + "_" + ad + ".nii.gz"
        save_nifti(patient['image'], output2, index)
#        print(output2)
        print(index)
        
    print(f'step {i} done')
elapsed = time.time()-start
print (f'Training time: {hms_string(elapsed)}')

0atient_generated_0 is saved
1atient_generated_1 is saved
2atient_generated_2 is saved
3atient_generated_3 is saved
4atient_generated_4 is saved
5atient_generated_5 is saved
6atient_generated_6 is saved
7atient_generated_7 is saved
8atient_generated_8 is saved
9atient_generated_9 is saved
10tient_generated_10 is saved
11tient_generated_11 is saved
12tient_generated_12 is saved
13tient_generated_13 is saved
14tient_generated_14 is saved
15tient_generated_15 is saved
16tient_generated_16 is saved
17tient_generated_17 is saved
18tient_generated_18 is saved
19tient_generated_19 is saved
20tient_generated_20 is saved
21tient_generated_21 is saved
22tient_generated_22 is saved
23tient_generated_23 is saved
24tient_generated_24 is saved
25tient_generated_25 is saved
26tient_generated_26 is saved
27tient_generated_27 is saved
28tient_generated_28 is saved
29tient_generated_29 is saved
30tient_generated_30 is saved
31tient_generated_31 is saved
32tient_generated_32 is saved
33tient_generated_33

In [7]:
#6min
name = "noise_1"
output_path = output_locaton + name


number_runs = 1
imglist = sorted(glob(input_path+"*.nii.gz"))
train_files =  [{"image": image} for image in zip(imglist)]

for i in range(number_runs):

    output = os.path.join(output_path)
    check_ds = Dataset(data=train_files, transform=noise_1)
    check_loader = DataLoader(check_ds, batch_size=1)
    check_data = first(check_loader)
    for index, patient in enumerate(check_loader):
        
        aa = str(train_files[index])
        ab = aa.split("\\")
        ac = ab[2].split(".")
        ad = ac[0]
        
        
        output2 = output + "/" + name + "_" + ad + ".nii.gz"
        save_nifti(patient['image'], output2, index)
#        print(output2)
        print(index)
elapsed = time.time()-start
print (f'Training time: {hms_string(elapsed)}')

0atient_generated_0 is saved
1atient_generated_1 is saved
2atient_generated_2 is saved
3atient_generated_3 is saved
4atient_generated_4 is saved
5atient_generated_5 is saved
6atient_generated_6 is saved
7atient_generated_7 is saved
8atient_generated_8 is saved
9atient_generated_9 is saved
10tient_generated_10 is saved
11tient_generated_11 is saved
12tient_generated_12 is saved
13tient_generated_13 is saved
14tient_generated_14 is saved
15tient_generated_15 is saved
16tient_generated_16 is saved
17tient_generated_17 is saved
18tient_generated_18 is saved
19tient_generated_19 is saved
20tient_generated_20 is saved
21tient_generated_21 is saved
22tient_generated_22 is saved
23tient_generated_23 is saved
24tient_generated_24 is saved
25tient_generated_25 is saved
26tient_generated_26 is saved
27tient_generated_27 is saved
28tient_generated_28 is saved
29tient_generated_29 is saved
30tient_generated_30 is saved
31tient_generated_31 is saved
32tient_generated_32 is saved
33tient_generated_33

In [8]:
#1min
start = time.time()
name = "shear_x1n"
output_path = output_locaton + name


number_runs = 1
imglist = sorted(glob(input_path+"*.nii.gz"))
train_files =  [{"image": image} for image in zip(imglist)]

for i in range(number_runs):

    output = os.path.join(output_path)
    check_ds = Dataset(data=train_files, transform=shear_x1n)
    check_loader = DataLoader(check_ds, batch_size=1)
    check_data = first(check_loader)
    for index, patient in enumerate(check_loader):
        
        aa = str(train_files[index])
        ab = aa.split("\\")
        ac = ab[2].split(".")
        ad = ac[0]
        
        
        output2 = output + "/" + name + "_" + ad + ".nii.gz"
        save_nifti(patient['image'], output2, index)
#        print(output2)
        print(index)
        
    print(f'step {i} done')
elapsed = time.time()-start
print (f'Training time: {hms_string(elapsed)}')

0atient_generated_0 is saved
1atient_generated_1 is saved
2atient_generated_2 is saved
3atient_generated_3 is saved
4atient_generated_4 is saved
5atient_generated_5 is saved
6atient_generated_6 is saved
7atient_generated_7 is saved
8atient_generated_8 is saved
9atient_generated_9 is saved
10tient_generated_10 is saved
11tient_generated_11 is saved
12tient_generated_12 is saved
13tient_generated_13 is saved
14tient_generated_14 is saved
15tient_generated_15 is saved
16tient_generated_16 is saved
17tient_generated_17 is saved
18tient_generated_18 is saved
19tient_generated_19 is saved
20tient_generated_20 is saved
21tient_generated_21 is saved
22tient_generated_22 is saved
23tient_generated_23 is saved
24tient_generated_24 is saved
25tient_generated_25 is saved
26tient_generated_26 is saved
27tient_generated_27 is saved
28tient_generated_28 is saved
29tient_generated_29 is saved
30tient_generated_30 is saved
31tient_generated_31 is saved
32tient_generated_32 is saved
33tient_generated_33

In [9]:
#1min
start = time.time()
name = "shear_x2n"
output_path = output_locaton + name


number_runs = 1
imglist = sorted(glob(input_path+"*.nii.gz"))
train_files =  [{"image": image} for image in zip(imglist)]

for i in range(number_runs):

    output = os.path.join(output_path)
    check_ds = Dataset(data=train_files, transform=shear_x2n)
    check_loader = DataLoader(check_ds, batch_size=1)
    check_data = first(check_loader)
    for index, patient in enumerate(check_loader):
        
        aa = str(train_files[index])
        ab = aa.split("\\")
        ac = ab[2].split(".")
        ad = ac[0]
        
        
        output2 = output + "/" + name + "_" + ad + ".nii.gz"
        save_nifti(patient['image'], output2, index)
#        print(output2)
        print(index)
        
    print(f'step {i} done')
elapsed = time.time()-start
print (f'Training time: {hms_string(elapsed)}')

0atient_generated_0 is saved
1atient_generated_1 is saved
2atient_generated_2 is saved
3atient_generated_3 is saved
4atient_generated_4 is saved
5atient_generated_5 is saved
6atient_generated_6 is saved
7atient_generated_7 is saved
8atient_generated_8 is saved
9atient_generated_9 is saved
10tient_generated_10 is saved
11tient_generated_11 is saved
12tient_generated_12 is saved
13tient_generated_13 is saved
14tient_generated_14 is saved
15tient_generated_15 is saved
16tient_generated_16 is saved
17tient_generated_17 is saved
18tient_generated_18 is saved
19tient_generated_19 is saved
20tient_generated_20 is saved
21tient_generated_21 is saved
22tient_generated_22 is saved
23tient_generated_23 is saved
24tient_generated_24 is saved
25tient_generated_25 is saved
26tient_generated_26 is saved
27tient_generated_27 is saved
28tient_generated_28 is saved
29tient_generated_29 is saved
30tient_generated_30 is saved
31tient_generated_31 is saved
32tient_generated_32 is saved
33tient_generated_33

In [10]:
#6min
start = time.time()
name = "elast_n"
output_path = output_locaton + name


number_runs = 1
imglist = sorted(glob(input_path+"*.nii.gz"))
train_files =  [{"image": image} for image in zip(imglist)]

for i in range(number_runs):

    output = os.path.join(output_path)
    check_ds = Dataset(data=train_files, transform=elast_n)
    check_loader = DataLoader(check_ds, batch_size=1)
    check_data = first(check_loader)
    for index, patient in enumerate(check_loader):
        
        aa = str(train_files[index])
        ab = aa.split("\\")
        ac = ab[2].split(".")
        ad = ac[0]
        
        
        output2 = output + "/" + name + "_" + ad + ".nii.gz"
        save_nifti(patient['image'], output2, index)
#        print(output2)
        print(index)
        
    print(f'step {i} done')
elapsed = time.time()-start
print (f'Training time: {hms_string(elapsed)}')

0atient_generated_0 is saved
1atient_generated_1 is saved
2atient_generated_2 is saved
3atient_generated_3 is saved
4atient_generated_4 is saved
5atient_generated_5 is saved
6atient_generated_6 is saved
7atient_generated_7 is saved
8atient_generated_8 is saved
9atient_generated_9 is saved
10tient_generated_10 is saved
11tient_generated_11 is saved
12tient_generated_12 is saved
13tient_generated_13 is saved
14tient_generated_14 is saved
15tient_generated_15 is saved
16tient_generated_16 is saved
17tient_generated_17 is saved
18tient_generated_18 is saved
19tient_generated_19 is saved
20tient_generated_20 is saved
21tient_generated_21 is saved
22tient_generated_22 is saved
23tient_generated_23 is saved
24tient_generated_24 is saved
25tient_generated_25 is saved
26tient_generated_26 is saved
27tient_generated_27 is saved
28tient_generated_28 is saved
29tient_generated_29 is saved
30tient_generated_30 is saved
31tient_generated_31 is saved
32tient_generated_32 is saved
33tient_generated_33

In [11]:
#1min
start = time.time()
name = "flip_xyn"
output_path = output_locaton + name


number_runs = 1
imglist = sorted(glob(input_path+"*.nii.gz"))
train_files =  [{"image": image} for image in zip(imglist)]

for i in range(number_runs):

    output = os.path.join(output_path)
    check_ds = Dataset(data=train_files, transform=flip_xyn)
    check_loader = DataLoader(check_ds, batch_size=1)
    check_data = first(check_loader)
    for index, patient in enumerate(check_loader):
        
        aa = str(train_files[index])
        ab = aa.split("\\")
        ac = ab[2].split(".")
        ad = ac[0]
        
        
        output2 = output + "/" + name + "_" + ad + ".nii.gz"
        save_nifti(patient['image'], output2, index)
#        print(output2)
        print(index)
        
    print(f'step {i} done')
elapsed = time.time()-start
print (f'Training time: {hms_string(elapsed)}')

0atient_generated_0 is saved
1atient_generated_1 is saved
2atient_generated_2 is saved
3atient_generated_3 is saved
4atient_generated_4 is saved
5atient_generated_5 is saved
6atient_generated_6 is saved
7atient_generated_7 is saved
8atient_generated_8 is saved
9atient_generated_9 is saved
10tient_generated_10 is saved
11tient_generated_11 is saved
12tient_generated_12 is saved
13tient_generated_13 is saved
14tient_generated_14 is saved
15tient_generated_15 is saved
16tient_generated_16 is saved
17tient_generated_17 is saved
18tient_generated_18 is saved
19tient_generated_19 is saved
20tient_generated_20 is saved
21tient_generated_21 is saved
22tient_generated_22 is saved
23tient_generated_23 is saved
24tient_generated_24 is saved
25tient_generated_25 is saved
26tient_generated_26 is saved
27tient_generated_27 is saved
28tient_generated_28 is saved
29tient_generated_29 is saved
30tient_generated_30 is saved
31tient_generated_31 is saved
32tient_generated_32 is saved
33tient_generated_33

In [12]:
#5min
start = time.time()
name = "flip_xye"
output_path = output_locaton + name


number_runs = 1
imglist = sorted(glob(input_path+"*.nii.gz"))
train_files =  [{"image": image} for image in zip(imglist)]

for i in range(number_runs):

    output = os.path.join(output_path)
    check_ds = Dataset(data=train_files, transform=flip_xye)
    check_loader = DataLoader(check_ds, batch_size=1)
    check_data = first(check_loader)
    for index, patient in enumerate(check_loader):
        
        aa = str(train_files[index])
        ab = aa.split("\\")
        ac = ab[2].split(".")
        ad = ac[0]
        
        
        output2 = output + "/" + name + "_" + ad + ".nii.gz"
        save_nifti(patient['image'], output2, index)
#        print(output2)
        print(index)
        
    print(f'step {i} done')
elapsed = time.time()-start
print (f'Training time: {hms_string(elapsed)}')

0atient_generated_0 is saved
1atient_generated_1 is saved
2atient_generated_2 is saved
3atient_generated_3 is saved
4atient_generated_4 is saved
5atient_generated_5 is saved
6atient_generated_6 is saved
7atient_generated_7 is saved
8atient_generated_8 is saved
9atient_generated_9 is saved
10tient_generated_10 is saved
11tient_generated_11 is saved
12tient_generated_12 is saved
13tient_generated_13 is saved
14tient_generated_14 is saved
15tient_generated_15 is saved
16tient_generated_16 is saved
17tient_generated_17 is saved
18tient_generated_18 is saved
19tient_generated_19 is saved
20tient_generated_20 is saved
21tient_generated_21 is saved
22tient_generated_22 is saved
23tient_generated_23 is saved
24tient_generated_24 is saved
25tient_generated_25 is saved
26tient_generated_26 is saved
27tient_generated_27 is saved
28tient_generated_28 is saved
29tient_generated_29 is saved
30tient_generated_30 is saved
31tient_generated_31 is saved
32tient_generated_32 is saved
33tient_generated_33

In [13]:
#6min
start = time.time()
name = "flip_xyen"
output_path = output_locaton + name


number_runs = 1
imglist = sorted(glob(input_path+"*.nii.gz"))
train_files =  [{"image": image} for image in zip(imglist)]

for i in range(number_runs):

    output = os.path.join(output_path)
    check_ds = Dataset(data=train_files, transform=flip_xyen)
    check_loader = DataLoader(check_ds, batch_size=1)
    check_data = first(check_loader)
    for index, patient in enumerate(check_loader):
        
        aa = str(train_files[index])
        ab = aa.split("\\")
        ac = ab[2].split(".")
        ad = ac[0]
        
        
        output2 = output + "/" + name + "_" + ad + ".nii.gz"
        save_nifti(patient['image'], output2, index)
#        print(output2)
        print(index)
        
    print(f'step {i} done')
elapsed = time.time()-start
print (f'Training time: {hms_string(elapsed)}')

0atient_generated_0 is saved
1atient_generated_1 is saved
2atient_generated_2 is saved
3atient_generated_3 is saved
4atient_generated_4 is saved
5atient_generated_5 is saved
6atient_generated_6 is saved
7atient_generated_7 is saved
8atient_generated_8 is saved
9atient_generated_9 is saved
10tient_generated_10 is saved
11tient_generated_11 is saved
12tient_generated_12 is saved
13tient_generated_13 is saved
14tient_generated_14 is saved
15tient_generated_15 is saved
16tient_generated_16 is saved
17tient_generated_17 is saved
18tient_generated_18 is saved
19tient_generated_19 is saved
20tient_generated_20 is saved
21tient_generated_21 is saved
22tient_generated_22 is saved
23tient_generated_23 is saved
24tient_generated_24 is saved
25tient_generated_25 is saved
26tient_generated_26 is saved
27tient_generated_27 is saved
28tient_generated_28 is saved
29tient_generated_29 is saved
30tient_generated_30 is saved
31tient_generated_31 is saved
32tient_generated_32 is saved
33tient_generated_33

In [16]:
#1min
start = time.time()
name = "ori_1"
output_path = output_locaton + name


number_runs = 1
imglist = sorted(glob(input_path+"*.nii.gz"))
train_files =  [{"image": image} for image in zip(imglist)]

for i in range(number_runs):

    output = os.path.join(output_path)
    check_ds = Dataset(data=train_files, transform=ori_1)
    check_loader = DataLoader(check_ds, batch_size=1)
    check_data = first(check_loader)
    for index, patient in enumerate(check_loader):
        
        aa = str(train_files[index])
        ab = aa.split("\\")
        ac = ab[2].split(".")
        ad = ac[0]
        
        
        output2 = output + "/" + name + "_" + ad + ".nii.gz"
        save_nifti(patient['image'], output2, index)
#        print(output2)
        print(index)
        
    print(f'step {i} done')
elapsed = time.time()-start
print (f'Training time: {hms_string(elapsed)}')

0atient_generated_0 is saved
1atient_generated_1 is saved
2atient_generated_2 is saved
3atient_generated_3 is saved
4atient_generated_4 is saved
5atient_generated_5 is saved
6atient_generated_6 is saved
7atient_generated_7 is saved
8atient_generated_8 is saved
9atient_generated_9 is saved
10tient_generated_10 is saved
11tient_generated_11 is saved
12tient_generated_12 is saved
13tient_generated_13 is saved
14tient_generated_14 is saved
15tient_generated_15 is saved
16tient_generated_16 is saved
17tient_generated_17 is saved
18tient_generated_18 is saved
19tient_generated_19 is saved
20tient_generated_20 is saved
21tient_generated_21 is saved
22tient_generated_22 is saved
23tient_generated_23 is saved
24tient_generated_24 is saved
25tient_generated_25 is saved
26tient_generated_26 is saved
27tient_generated_27 is saved
28tient_generated_28 is saved
29tient_generated_29 is saved
30tient_generated_30 is saved
31tient_generated_31 is saved
32tient_generated_32 is saved
33tient_generated_33

In [25]:
#import os

# Get the path to the folder
#folder_path = "C:/Users/85298/AI/CT_image/202304/224_160_48/elast_1/"

# Get all the files in the folder
#files = os.listdir(folder_path)

# Rename all the files
#for file in files:
    
#    x = file.split("_") 
#    new_file_name = x[0] + "_1_" + x[1] + "_ct.nii.gz"
    
#    print (new_file_name)
#    os.rename(os.path.join(folder_path, file), os.path.join(folder_path, new_file_name))

elast_1_1000_ct.nii.gz
elast_1_1001_ct.nii.gz
elast_1_1002_ct.nii.gz
elast_1_1003_ct.nii.gz
elast_1_1004_ct.nii.gz
elast_1_1005_ct.nii.gz
elast_1_1006_ct.nii.gz
elast_1_1007_ct.nii.gz
elast_1_1008_ct.nii.gz
elast_1_1009_ct.nii.gz
elast_1_1010_ct.nii.gz
elast_1_1011_ct.nii.gz
elast_1_1012_ct.nii.gz
elast_1_1013_ct.nii.gz
elast_1_1014_ct.nii.gz
elast_1_1015_ct.nii.gz
elast_1_1016_ct.nii.gz
elast_1_1017_ct.nii.gz
elast_1_1018_ct.nii.gz
elast_1_1019_ct.nii.gz
elast_1_1020_ct.nii.gz
elast_1_1022_ct.nii.gz
elast_1_1023_ct.nii.gz
elast_1_1024_ct.nii.gz
elast_1_1025_ct.nii.gz
elast_1_1026_ct.nii.gz
elast_1_1027_ct.nii.gz
elast_1_1028_ct.nii.gz
elast_1_1029_ct.nii.gz
elast_1_2000_ct.nii.gz
elast_1_2001_ct.nii.gz
elast_1_2002_ct.nii.gz
elast_1_2003_ct.nii.gz
elast_1_2004_ct.nii.gz
elast_1_2005_ct.nii.gz
elast_1_2006_ct.nii.gz
elast_1_2007_ct.nii.gz
elast_1_2008_ct.nii.gz
elast_1_2009_ct.nii.gz
elast_1_2010_ct.nii.gz
elast_1_2011_ct.nii.gz
elast_1_2012_ct.nii.gz
elast_1_2013_ct.nii.gz
elast_1_300